<b>Problem statement</b> : Lot of money needs to be spent on advertisement of music and customers might not buy it.

<b>Objective</b> : We have data from a music company site where we have a log of every artist’s music the user has downloaded . We need to build a system that recommends new music to user based on the data available.

<b>Possible actions to be taken</b> :
First we need to find out support,confidence and lift of the data values

(Support  is the probability of buying both the products

Confidence is the probability of buying product B given that product A has already been bought.

Lift tells if the 2 products are independent or dependent and how they are related.)

Here we have multiple products or artists .We need to figure out which music is related to which one to recommend it to the user. 
We will use Apyroi algorithm to generate association rules.
With the help of association rule,we can determine how strongly or how weakly 2 products are connected.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori,association_rules
df_raw=pd.read_csv("lastfm.csv")
pd.options.display.max_columns = 25
pd.options.display.max_rows = None

In [2]:
def basic_data_report(df_report,drop_duplicate):
    BOLD = '\033[1m'
    END = '\033[0m'
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numeric_variables = df_report.select_dtypes(include = 'int')
    categorical_variables = df_report.select_dtypes(include = 'object')
    print(BOLD + "rows,columns :"+END,df_report.shape,"\n")
    print(BOLD +"Column & Data Type \n"+END,df_report.dtypes,"\n")
    print(BOLD +"Columns and Corresponding nullValues\n"+END,df_report.isnull().sum(),"\n")
    print(BOLD + "Sample \n"+END,df_report.sample(5),"\n")
    print(BOLD + "Number of Duplicated rows \n"+END,df_report.duplicated().sum(),"\n")
    print(BOLD + "Describing the numeric features \n"+END,df_report.describe(),"\n")
    print(BOLD+"Describing Categorical variables"+END)
    for idx,variable in enumerate(categorical_variables.columns):
        print(BOLD+variable+END,"\n",categorical_variables[variable].value_counts(),"\n")
    if drop_duplicate:
        global df
        df = df_report.drop_duplicates()
        print("rows after removing duplicates",df.shape[0])

In [3]:
basic_data_report(df_raw,True)

rows,columns : (289955, 4) 

Column & Data Type 
 user        int64
artist     object
sex        object
country    object
dtype: object 

Columns and Corresponding nullValues
 user       0
artist     0
sex        0
country    0
dtype: int64 

Sample 
          user         artist sex         country
231566  15730     pink floyd   m          Turkey
244297  16620        beyoncé   m  United Kingdom
162404  11055          björk   m         Germany
17195    1129  john williams   f         Romania
224117  15217         m.i.a.   m          Canada 

Number of Duplicated rows 
 2 

Describing the numeric features 
                 user
count  289955.000000
mean     9852.460447
std      5692.355041
min         1.000000
25%      4935.000000
50%      9838.000000
75%     14769.000000
max     19718.000000 

Describing Categorical variables
artist 
 radiohead                                       2704
the beatles                                     2668
coldplay                                       

country 
 United States                                   59558
United Kingdom                                  27638
Germany                                         24251
Poland                                          17111
Sweden                                          12379
Brazil                                          11922
Russian Federation                              11676
Finland                                         10157
Netherlands                                      9673
Spain                                            9322
Canada                                           6928
Australia                                        6407
France                                           5962
Italy                                            5717
Turkey                                           4762
Norway                                           4641
Mexico                                           3803
Czech Republic                                   3752
Belgium           

In [4]:
basket = df.groupby(['user','artist'])['user'].count().unstack()
basket.head()

artist,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,[unknown],a day to remember,a fine frenzy,...,volbeat,weezer,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
user,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def ConvertToBinary(x):
    if x>0:
        return 1
    else :
        return 0

In [6]:
df_binary = basket.applymap(ConvertToBinary)
df_binary.shape

(15000, 1004)

In [ ]:
freq_items = apriori(df_binary, min_support = .01, use_colnames = True) 
freq_items.head()

In [ ]:
rules = association_rules(freq_items,metric = 'confidence',min_threshold = 0.5)
rules.head()

In [ ]:
rules[rules["lift"]>5].sort_values(by='confidence',ascending = False)